In [ ]:
from types import SimpleNamespace
import os
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import requests
import json
SWAPI_ROOT = "https://swapi.dev/api/"


In [ ]:
# Connect to an existing database
# Creates a new database session and returns a new connection instance.
conn = psycopg2.connect(
    host = "db",
    dbname = "postgres",
    user = "postgres",
    password = "example",
    port = 5432
)

#dbname – the database name (database is a deprecated alias)
#user – user name used to authenticate
#password – password used to authenticate
#host – database host address (defaults to UNIX socket if not provided)
#port – connection port number 

In [ ]:
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

In [ ]:
# Execute a command: this creates a new database
with conn.cursor() as cur:
    cur.execute("CREATE DATABASE star_wars;")

In [ ]:
conn.close()

In [141]:
conn = psycopg2.connect(
    host = "db",
    dbname = "star_wars",
    user = "postgres",
    password = "example",
    port = 5432
)

In [142]:
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

In [157]:
# Open a cursor to perform database operations
cur = conn.cursor()

In [167]:
   resource_types = [
        "films",
        "people",
        "planets",
        "species",
        "starships",
        "vehicles",
    ]

In [168]:
def create_table_from_schema(schema_name):

    def import_schema(schema):
        with open(f"work/schemas/{schema}.json", 'r') as fp:
            result = json.load(fp)
            return result
        fp.close()

    def convert_python_postgres(dictionary):
        for element in dictionary['properties']:
            if dictionary['properties'][element]['type'] == "string":
                dictionary['properties'][element]['type'] = "VARCHAR"
        
            if dictionary['properties'][element]['type'] == "array":
                dictionary['properties'][element]['type'] = "VARCHAR[]"
        
        return dictionary

    def create_table(dictionary):
        try:
            table_name = dictionary['title']
            cur.execute("CREATE TABLE IF NOT EXISTS {} (id SERIAL PRIMARY KEY);".format(table_name))
            for element in dictionary['properties']:
                cur.execute("ALTER TABLE {} ADD COLUMN {} {};".format(table_name,element,dictionary['properties'][element]['type']))

        except (Exception, psycopg2.Error) as error:
            print(f"Error-code: {error}")


    result = import_schema(schema_name)
    result = convert_python_postgres(result)
    create_table(result)
   
for resource_type in resource_types:
    create_table_from_schema(resource_type)

conn.commit()

In [182]:
def fill_tables():
    def get_swapi_resource(path: str):
        r = requests.get(SWAPI_ROOT + path)
        if r.ok:
            return r.json()
        else:
            return None

   
    for resource_type in resource_types:
        resource = get_swapi_resource(resource_type)
        result = list()
        counter = 0
        i = 1
        while counter < resource["count"]:
            response = get_swapi_resource(f"{resource_type}/{i}")
            i += 1
            if response != None:
                counter += 1
                result.append(response)
        for r in result:
            keys = r.keys()
            columns = ','.join(keys)
            values = ','.join(['%({})s'.format(k) for k in keys])
            insert = 'insert into {0} ({1}) values ({2})'.format(resource_type,columns, values)
            cur.execute(cur.mogrify(insert, r))
            

fill_tables()
conn.commit()